In [1]:
import torch
import dataset as my_dataset
import pandas as pd

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [3]:
path = "/home/host/Documents/JupyterProjects/Test_ML_DBA/dataset_test_task/"

loaded_data = my_dataset.DataLoad(path)
loaded_data.data.head()

,image,label
0,30900_5_010.jpg,010
1,267155_9_010.jpg,010
2,125867_3_000.jpg,000
3,10762_10_010.jpg,010
4,68924_5_000.jpg,000


In [4]:
data = loaded_data.data
data.shape

(6000, 2)

In [5]:
# Хера себе дисбаланс

# [000] = 2186
# [001] = 545
# [010] = 2008
# [011] = 5
# [100] = 1226
# [101] = 8
# [110] = 22
# [111] = 0 - нет данных

# Как-то нужно размножить классы [011], [101], [110]
main_df = my_dataset.augmented_dataset(path, data)
print('Размер набора данных', main_df.shape)

Размер набора данных (6620, 2)


In [6]:
x_train, x_valid, x_test = my_dataset.split_data(main_df)

x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

print('Размер тренировочного набора данных', x_train.shape)
print('Размер валидационного набора данных', x_valid.shape)
print('Размер тестового набора данных', x_test.shape)

Размер тренировочного набора данных (5296, 2)
Размер валидационного набора данных (662, 2)
Размер тестового набора данных (662, 2)


In [7]:
from dataset import CustomDataset

train_dataset = CustomDataset(data_frame=x_train)
valid_dataset = CustomDataset(data_frame=x_valid)
test_dataset = CustomDataset(data_frame=x_test)

In [8]:
from torch.utils.data import DataLoader

batch_size = 64 # размер батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
import model
from train import train_model

net = model.CNN().to(device)

loss_history, best_model, y_pred_valid = train_model(train_loader, valid_loader, net, epochs=5, device=device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 2., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 2., 0., 0., 

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
from train import test_model

y_pred_test = test_model(model=best_model, loader_test=test_loader, device=device)